In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
import joblib
import os

# Load CSV
df = pd.read_csv("data/VMC_Dashboard_Corrected_3000_Entries_With_Date_Time.csv")

# Drop unnecessary columns
df = df.drop(columns=['Date', 'Time', 'Job ID', 'Operator ID', 'Tool Number'])

# Keep only 'OK' and 'NG'
df = df[df['Part Status'].isin(['OK', 'NG'])]

# Encode target
label_encoder = LabelEncoder()
df['Part Status'] = label_encoder.fit_transform(df['Part Status'])  # OK=1, NG=0

print("🎯 Class Distribution BEFORE balancing:")
print(df['Part Status'].value_counts())

# Split into OK and NG
ok_df = df[df['Part Status'] == 1]
ng_df = df[df['Part Status'] == 0]

# Resample NG to match OK
ng_upsampled = resample(ng_df, replace=True, n_samples=len(ok_df), random_state=42)
balanced_df = pd.concat([ok_df, ng_upsampled])

print("✅ Class Distribution AFTER balancing:")
print(balanced_df['Part Status'].value_counts())

# Define features
features = ['Cycle Time (s)', 'Job Count', 'Machine Utilization (%)',
            'Spindle Speed (RPM)', 'Feed Rate (mm/min)', 'Tool Wear (%)',
            'Vibration Level (mm/s)', 'Spindle Temperature (°C)', 'Power Consumption (kWh)']

X = balanced_df[features]
y = balanced_df['Part Status']

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print("📉 Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("📊 Classification Report:\n", classification_report(y_test, y_pred))

# Save model
if not os.path.exists("model"):
    os.mkdir("model")

joblib.dump(model, "model/part_status_predictor.pkl")
print("✅ Model saved successfully!")


🎯 Class Distribution BEFORE balancing:
Part Status
1    2535
0     149
Name: count, dtype: int64
✅ Class Distribution AFTER balancing:
Part Status
1    2535
0    2535
Name: count, dtype: int64
📉 Confusion Matrix:
 [[495   0]
 [  0 519]]
📊 Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       495
           1       1.00      1.00      1.00       519

    accuracy                           1.00      1014
   macro avg       1.00      1.00      1.00      1014
weighted avg       1.00      1.00      1.00      1014

✅ Model saved successfully!
